In [1]:
from firedrake import *
from pyadjoint import *
import numpy as np



# 1. Configuración inicial
mesh = UnitSquareMesh(10, 10)
coords = mesh.coordinates.copy()

# 2. Espacio para el control (deformación)
V_control = VectorFunctionSpace(mesh, "CG", 1)
control = Function(V_control, name="Control")
m = Control(control)

# 3. Mecanismo de deformación CORREGIDO
def deformed_coordinates():
    """Crea dependencia explícita en el grafo computacional"""
    return coords + control

# 4. Problema directo con dependencia explícita
def forward(control):
    # Asegurar que estamos grabando operaciones
    with stop_annotating():
        mesh.coordinates.assign(deformed_coordinates())
    
    V_u = VectorFunctionSpace(mesh, "CG", 1)
    u = Function(V_u, name="State")
    
    # Elasticidad lineal
    E = Constant(1e5)
    nu = Constant(0.3)
    mu = E/(2*(1+nu))
    lmbda = E*nu/((1+nu)*(1-2*nu))
    
    def sigma(u):
        return 2*mu*sym(grad(u)) + lmbda*tr(sym(grad(u)))*Identity(2)
    
    v = TestFunction(V_u)
    f = Constant((0, -0.01))
    F = inner(sigma(u), grad(v)) * dx - inner(f, v) * dx
    
    bc = DirichletBC(V_u, Constant((0, 0)), (1, 2))
    
    solve(F == 0, u, bcs=bc, 
          solver_parameters={
              "snes_type": "newtonls",
              "snes_max_it": 100,
              "ksp_type": "preonly",
              "pc_type": "lu"
          })
    
    return u

# 5. Ejecutar el problema directo con anotación
u = forward(control)

# 6. Funcional con dependencia explícita
J = assemble(inner(grad(u), grad(u)) * dx)
Jhat = ReducedFunctional(AdjFloat(J), m)

# 7. Verificación con perturbación
perturbation = Function(V_control)
perturbation.dat.data[:] = 0.001 * np.random.rand(*perturbation.dat.data.shape)
# print("Verificación por perturbación:")
# print(f"J(m) = {Jhat(m)}")
# print(f"J(m + ε) = {Jhat(m + perturbation)}")
# print(f"Norma de la derivada: {norm(Jhat.derivative())}")

# Prueba de derivada
print("Derivative norm:", norm(Jhat.derivative()))

Derivative norm: 0.0


In [4]:
Tape.visualise("/home/brandon/Shape_Optimization/archivo_3.png")

AttributeError: 'str' object has no attribute '_tf_add_blocks'